In [7]:
# import pandas as pd

# reviews = pd.read_csv("austin_yelp_reviews.csv")
# unique = reviews['Restaurant Name'].unique()
# print(unique)
# pd.DataFrame(unique).to_csv('austin_restaurants.csv', index=False)

In [8]:
import pandas as pd

austin_reviews = pd.read_csv("austin_yelp_reviews.csv")
michelin_reviews = pd.read_csv("Michelin Reviews.csv")
non_michelin_reviews = pd.read_csv("yelp_reviews_non_michelin.csv")

In [13]:
michelin_reviews['Review Length'] = michelin_reviews['Review'].apply(lambda x: len(x.split()))
non_michelin_reviews['Review Length'] = non_michelin_reviews['Review'].apply(lambda x: len(x.split()))
print(michelin_reviews['Review Length'].mean())
print(non_michelin_reviews['Review Length'].mean())

169.21166343632424
122.57221761944214


In [23]:
austin_reviews['Review Length'] = austin_reviews['Review'].apply(lambda x: len(x.split()))
avg_review_length_per_restaurant = austin_reviews.groupby('Restaurant Name')['Review Length'].mean()
print(avg_review_length_per_restaurant.sort_values(ascending=False).head(50))


Restaurant Name
The Driskill Grill                  199.020000
Conje                               176.200000
Franklin Barbecue                   175.770000
dipdipdip tatsuya                   175.030000
Arlo Grey                           172.850000
Sushi Endo                          168.750000
Otoko                               168.710000
Juniper                             162.920000
Tsuke Edomae                        158.289157
Garrison                            157.220000
The Guest House                     157.190000
Uchiko                              155.950000
Uchi                                155.950000
Lenoir                              155.410000
Hestia                              153.500000
Intero                              152.580000
Kemuri Tatsu-Ya                     152.376238
Patika Coffee                       152.250000
Craft Omakase                       151.735294
Contigo                             148.363636
Honey Moon Spirit Lounge            146.4300

In [ ]:
# review length, sentiment score, michelin principles
# use z-score

In [25]:
import spacy
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


# Load the dataset
#file_path = 'PUT YOUR FILE HERE  # Replace with the actual file path
reviews_df = pd.read_csv('yelp_reviews_non_michelin.csv')
# Rename columns if necessary
reviews_df.columns = ['Restaurant Name', 'Location', 'Review']
# Load spaCy's language model for Named Entity Recognition
nlp = spacy.load('en_core_web_sm')
# Function to extract various relevant entities
def extract_relevant_entities(text):
    doc = nlp(text)
    # Define the entities we are interested in
    relevant_entities = {
        'PERSON': [],   # Chefs, Staff, etc.
        'FOOD': [],     # Food-related entities
        'ORG': [],      # Organizations
        'PRODUCT': [],  # Food-related products
        'FAC': []      # Facilites
    }
    # Extract entities and categorize them
    for ent in doc.ents:
        if ent.label_ in relevant_entities.keys():
            relevant_entities[ent.label_].append(ent.text)
    return relevant_entities
# Apply the function to extract relevant entities from reviews
reviews_df['entities'] = reviews_df['Review'].apply(extract_relevant_entities)
# Display first few rows to see extracted entities
print(reviews_df[['Restaurant Name', 'entities']].head())
# Optional: remove reviews with no entities (if you want to filter by entity presence)
reviews_with_entities = reviews_df[reviews_df['entities'].apply(lambda x: any(x.values()))]
# Topic Modeling based on relevant entities
def perform_topic_modeling(corpus, num_topics=5, num_words=10):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(corpus)
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(X)
    # Display topics and top words
    feature_names = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        top_features = [feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]
        print(f"Topic {topic_idx + 1}: {', '.join(top_features)}")
# Create a new column by joining extracted entities to form a corpus for topic modeling
reviews_df['entity_corpus'] = reviews_df['entities'].apply(lambda x: ' '.join([item for sublist in x.values() for item in sublist]))
# Filter for reviews with at least some entities
filtered_reviews = reviews_df[reviews_df['entity_corpus'] != '']
# Perform topic modeling on extracted entities
perform_topic_modeling(filtered_reviews['entity_corpus'], num_topics=5, num_words=10)

  Restaurant Name                                           entities
0          Lucali  {'PERSON': [], 'FOOD': [], 'ORG': [], 'PRODUCT...
1          Lucali  {'PERSON': [], 'FOOD': [], 'ORG': [], 'PRODUCT...
2          Lucali  {'PERSON': ['David Beckham'], 'FOOD': [], 'ORG...
3          Lucali  {'PERSON': [], 'FOOD': [], 'ORG': [], 'PRODUCT...
4          Lucali  {'PERSON': [], 'FOOD': [], 'ORG': ['NY'], 'PRO...
Topic 1: benedict, covid, goody, patty, la, wow, arancini, thai, works, cafe
Topic 2: michelin, mac, bbq, chef, cheese, tacos, mary, burger, bartaco, pepe
Topic 3: al, el, food, steak, ny, dente, michelin, sushi, bourbon, truffle
Topic 4: taco, dessert, joe, al, michelin, miami, spice, valet, tacos, lobster
Topic 5: chicken, crispy, fried, strawberry, shrimp, rice, salad, dan, brussel, yelp


In [5]:
!pip install lda


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.0/350.0 kB 5.0 MB/s eta 0:00:00


In [8]:
#!/usr/bin/python3.6
#Install LDA library if not already installed
# pip3.6 install --user lda
# the input file is yelp_reviews.xslx
# there are two output files: topic_word_dist.xlsx and document_topic_dist.xlsx
# the script prompts for the name of the columns -- Restaurant_name and Restaurant_review in the yelp_reviews.xslx file
import os, csv, nltk, lda
import pandas as pd
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import PunktSentenceTokenizer,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

reviews_df=pd.read_csv("Michelin Reviews.csv",encoding='utf8')
#checking for nulls if present any
print("Number of rows with any of the empty columns:")
print(reviews_df.isnull().sum().sum())

reviews_df=reviews_df.dropna()
restaurant_name = input('provide the column name for id: ')
restaurant_review = input('provide the column name for text: ')
ntopics= input('Provide the number of latent topics: ')

word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))

def tokenize_text(version_desc):
  lowercase=version_desc.lower()
  text = wordnet_lemmatizer.lemmatize(lowercase)
  tokens = word_tokenizer.tokenize(text)
  return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=list(stopwords_nltk),decode_error='ignore')
total_features_words = vec_words.fit_transform(reviews_df[restaurant_review])
print(total_features_words.shape)

model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_features_words)
topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
reviews_df=reviews_df.join(doc_topic)
restaurant=pd.DataFrame()

for i in range(int(ntopics)):
  topic="topic_"+str(i)
  restaurant[topic]=reviews_df.groupby([restaurant_name])[i].mean()

restaurant=restaurant.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names_out()
topics1=topics.transpose()
print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics1.to_excel("topic_word_dist.xlsx")
restaurant.to_excel("document_topic_dist.xlsx",index=False)
print ("Document topic distribution written in file document_topic_dist.xlsx ")
print(restaurant)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Number of rows with any of the empty columns:
0
provide the column name for id: Restaurant Name
provide the column name for text: Review
Provide the number of latent topics: 3


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn(


(17542, 42735)
Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 
         Restaurant Name   topic_0   topic_1   topic_2
0                7 Adams  0.659648  0.039831  0.300521
1             Acquerello  0.689485  0.041233  0.269283
2                Addison  0.771234  0.139097  0.089670
3                   Albi  0.686030  0.024457  0.289513
4                 Alinea  0.840562  0.069460  0.089978
..                   ...       ...       ...       ...
191  Victoria & Albert's  0.835096  0.143065  0.021839
192             Wakuriya  0.714106  0.169360  0.116535
193              Yoshino  0.590203  0.362281  0.047516
194               n/naka  0.680171  0.231698  0.088131
195                  odo  0.641789  0.207953  0.150258

[196 rows x 4 columns]
